### binary search 
---



In [ ]:
def binary_search(array, element):
  mid = 0
  start = 0
  end = len(array)
  step = 0
  
  while (start <= end):
    step += 1
    mid = (start + end) // 2
    if element == array[mid]:
      return mid
    if elemnet < array[mid]:
      end = mid - 1
    else: start = mid + 1
  return -1 

In [10]:
import numpy as np
from PIL import Image
import json
from glob import glob
from pathlib import Path

import cv2
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import imutils  # 멀까
import time
from datetime import timedelta  # 멀까
from collections import deque, defaultdict
import re
import matplotlib.image as mpimg  # 멀까
from collections import OrderedDict
from skimage import io, transform  # 멀까
from math import *
import xml.etree.ElementTree as ET  # 멀까


# torch
import torch
import torchvision  # 토치비전엔 모델들이 보통 있던거 같은데
import torch.nn as nn  # nn.module , nn.squential  등등
import torch.optim as optim  # 아담 모멘텀 아다그레이드 등등
import torch.nn.functional as F  # Relu 같은게 있겟지
import torchvision.transforms as transforms  # augmentation 할수 있는 것들
import torchvision.transforms.functional as TF  # 정확히 머가 있는지 모르겟다
# dataset이 있고 , model 이있고 transform 이있다
from torchvision import datasets, models, transforms
# torch utils data  에는 Dataset과 Dataloader가 들어있다
from torch.utils.data import Dataset, DataLoader


In [3]:
class Attribute(dict):
  def __init__(self):
    self.__dict__ = self
    
class ConfigTree:
  def __init__(self):
    self.DATASET = Attribute()
    self.SYSTEM = Attribute()
    self.Train = Attribute()
    self.MODEL = Attribute()
    self.KD = Attribute()
    

class image_Dataset_(Dataset):
  def __init__(Self, root, cfg=None, transform=None):
    super(image_Dataset_, self).__init__()
    self.root = root
    self.cfg = cfg
    self.transform = transform
    self.images = sorted(glob(self.root + '/*.jpg'))
    self.annotations = sorted(glob(self.root + '/*.metadata'))
    
  def __getitem__(self, idx):
    label = self.label2logit(self.get_gt(self.annotations[idx]))
    image = image.open(self.images[idx])
    if self.transfrom:
      image = self.transfrom(image)
    
  def __len__(self):
    return len(self.images)
  
  def get_gt(self, meta_file):
    gt_class = None
    meta_file = Path(meta_file)
    f = open(meta_file, 'r')
    gt_class = f.readline()[22].split()[1].strip()
    return gt_class
    
  def label2logit(self, label):
    str_dict = {
      'female': 0,
      'male': 1
    }
    return label2logit['label']

def get_loader(dataset, cfg, suffle=True):
  return DataLoader(dataset,batch_size=cfg.TRAIN.BATCH_SIZE,suffle=suffle)
  


In [4]:
def get_augmentation(size=224, use_flip=True, use_color_jitter=False, use_gray_scale=False, use_normalize=False):
  resize_crop = transforms.RandomResizedCrop(size=size)
  random_flip = transforms.RandomHorizontalFlip(p=0.5)
  color_jitter = transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8)
  gray_scale = transforms.RandomGrayscale(p=0.2)
  normalize = transforms.Normalize(
      (0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
  to_tensor = transforms.ToTensor()
  transforms_array = np.array([resize_crop, random_flip, color_jitter, gray_scale, to_tensor, normalize])
  transforms_mask = np.array([True, use_flip, use_color_jitter, use_gray_scale, True, use_normalize])
  
  transform = transforms.Compose(transforms_array[transforms_mask])
  return transform
  


In [11]:
class Student(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = models.ResNet(models.resnet.BasicBlock, [2, 2, 2, 2])
    self.model.fc = nn.Linear(512, 2)

  def forward(self, x):
    x = self.model(x)
    return x
    

In [12]:
Student()

Student(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [13]:
### 모델 확인하기 

from torchsummary import summary

try:
  assert Student().model.fc.out_features == 2
  summary(Student(), (3, 224, 224), device='cpu')
  
except:
  print('student Network를 다시만드세요')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,